In [1]:
import numpy as np
import pandas as pd
!pip install geopandas
import geopandas as gpd
from datetime import datetime,timedelta

import bokeh.palettes as bp
from bokeh.plotting import figure,curdoc
from bokeh.transform import linear_cmap
from bokeh.layouts import column, row
from bokeh.models import (CDSView, 
						HoverTool,ColorBar,
						GeoJSONDataSource, 
						Patches,
						RadioButtonGroup,
						DateSlider,
						Button)

In [2]:
demo_url = 'https://raw.githubusercontent.com/daenuprobst/covid19-cases-switzerland/master/demographics.csv'
local_url = 'https://raw.githubusercontent.com/daenuprobst/covid19-cases-switzerland/master/covid_19_cases_switzerland_standard_format.csv'
case_url = 'https://raw.githubusercontent.com/daenuprobst/covid19-cases-switzerland/master/covid19_cases_switzerland_openzh-phase2.csv'

In [3]:
import pandas as pd
demo_raw = pd.read_csv(demo_url)
local_raw = pd.read_csv(local_url)
case_raw = pd.read_csv(case_url, low_memory=False,
                 parse_dates=["Date"])

In [4]:
demo_raw.drop(demo_raw.index[[26]])
demo_raw.head()

,Canton,Population,SettlementAreaHa,SettlementAreaKm2,Density,O65,O65P,Beds,BedsPerCapita
0,ZH,1520968,37796.0,377.96,4024.150704,0.170,258565,4472,0.002940
1,BE,1034977,41197.0,411.97,2512.263029,0.208,215275,3053,0.002950
2,VD,799145,29940.0,299.40,2669.154977,0.164,131060,2268,0.002838
3,AG,678207,23854.0,238.54,2843.158380,0.177,120043,1450,0.002138
4,SG,507697,19408.0,194.08,2615.916117,0.183,92909,1565,0.003083


In [5]:
local_raw.head()

,date,country,abbreviation_canton,name_canton,lat,long,hospitalized_with_symptoms,intensive_care,total_hospitalized,home_confinment,total_currently_positive_cases,new_positive_cases,recovered,deaths,total_positive_cases,tests_performed
0,2020-03-06T00:00:00,Switzerland,AG,Aargau,47.449406,8.327495,NaN,NaN,NaN,NaN,10.0,0.0,NaN,NaN,NaN,NaN
1,2020-03-06T00:00:00,Switzerland,AI,Appenzell Innerrhoden,47.328414,9.409647,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,2020-03-06T00:00:00,Switzerland,AR,Appenzell Ausserrhoden,47.382710,9.271860,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN
3,2020-03-06T00:00:00,Switzerland,BE,Bern,46.916667,7.466667,NaN,NaN,NaN,NaN,20.0,0.0,NaN,0.0,NaN,NaN
4,2020-03-06T00:00:00,Switzerland,BL,Baselland,47.482779,7.742975,NaN,NaN,NaN,NaN,9.0,0.0,NaN,NaN,NaN,NaN


In [6]:
# Extract unique 'abbreviation_canton','lat','long' combinations from local_raw
canton_point = local_raw[['abbreviation_canton','lat','long']]
canton_point.shape

(572, 3)

In [7]:
canton_point = canton_point.drop_duplicates()
canton_point.shape

(26, 3)

In [8]:
canton_point.head()

,abbreviation_canton,lat,long
0,AG,47.449406,8.327495
1,AI,47.328414,9.409647
2,AR,47.382710,9.271860
3,BE,46.916667,7.466667
4,BL,47.482779,7.742975


In [9]:
case_raw.head()

,Date,AG,AI,AR,BE,BL,BS,FR,GE,GL,...,VS_diff_pc,ZG_diff,ZG_pc,ZG_diff_pc,ZH_diff,ZH_pc,ZH_diff_pc,CH_diff,CH_pc,CH_diff_pc
0,2020-05-31,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,0.0,NaN,NaN,0.000000,NaN,NaN,0.000000e+00,NaN
1,2020-06-01,0.0,NaN,NaN,0.0,1.0,0.0,0.0,1.0,NaN,...,0.000003,0.0,0.0,0.0,0.0,0.000000,0.000000,3.0,3.512301e-07,3.512301e-07
2,2020-06-02,0.0,NaN,0.0,1.0,1.0,0.0,2.0,4.0,0.0,...,0.000003,0.0,0.0,0.0,2.0,0.000001,0.000001,17.0,2.341534e-06,1.990304e-06
3,2020-06-03,1.0,0.0,0.0,2.0,2.0,1.0,3.0,6.0,0.0,...,0.000000,0.0,0.0,0.0,3.0,0.000003,0.000002,18.0,4.448914e-06,2.107380e-06
4,2020-06-04,3.0,NaN,0.0,4.0,2.0,1.0,5.0,8.0,0.0,...,0.000006,0.0,0.0,0.0,8.0,0.000009,0.000005,24.0,7.258755e-06,2.809841e-06


In [10]:
print(case_raw.columns.tolist())

['Date', 'AG', 'AI', 'AR', 'BE', 'BL', 'BS', 'FR', 'GE', 'GL', 'GR', 'JU', 'LU', 'NE', 'NW', 'OW', 'SG', 'SH', 'SO', 'SZ', 'TG', 'TI', 'UR', 'VD', 'VS', 'ZG', 'ZH', 'CH', 'AG_diff', 'AG_pc', 'AG_diff_pc', 'AI_diff', 'AI_pc', 'AI_diff_pc', 'AR_diff', 'AR_pc', 'AR_diff_pc', 'BE_diff', 'BE_pc', 'BE_diff_pc', 'BL_diff', 'BL_pc', 'BL_diff_pc', 'BS_diff', 'BS_pc', 'BS_diff_pc', 'FR_diff', 'FR_pc', 'FR_diff_pc', 'GE_diff', 'GE_pc', 'GE_diff_pc', 'GL_diff', 'GL_pc', 'GL_diff_pc', 'GR_diff', 'GR_pc', 'GR_diff_pc', 'JU_diff', 'JU_pc', 'JU_diff_pc', 'LU_diff', 'LU_pc', 'LU_diff_pc', 'NE_diff', 'NE_pc', 'NE_diff_pc', 'NW_diff', 'NW_pc', 'NW_diff_pc', 'OW_diff', 'OW_pc', 'OW_diff_pc', 'SG_diff', 'SG_pc', 'SG_diff_pc', 'SH_diff', 'SH_pc', 'SH_diff_pc', 'SO_diff', 'SO_pc', 'SO_diff_pc', 'SZ_diff', 'SZ_pc', 'SZ_diff_pc', 'TG_diff', 'TG_pc', 'TG_diff_pc', 'TI_diff', 'TI_pc', 'TI_diff_pc', 'UR_diff', 'UR_pc', 'UR_diff_pc', 'VD_diff', 'VD_pc', 'VD_diff_pc', 'VS_diff', 'VS_pc', 'VS_diff_pc', 'ZG_diff', 'Z

In [11]:
# Create a date list from case_raw and convert to datatime form
dates = case_raw['Date']
dates

0     2020-05-31
1     2020-06-01
2     2020-06-02
3     2020-06-03
4     2020-06-04
         ...    
196   2020-12-13
197   2020-12-14
198   2020-12-15
199   2020-12-16
200   2020-12-17
Name: Date, Length: 201, dtype: datetime64[ns]

In [12]:
dates = np.array(dates, dtype='datetime64[D]')

In [13]:
dates = np.datetime_as_string(dates)

In [14]:
dates

array(['2020-05-31', '2020-06-01', '2020-06-02', '2020-06-03',
       '2020-06-04', '2020-06-05', '2020-06-06', '2020-06-07',
       '2020-06-08', '2020-06-09', '2020-06-10', '2020-06-11',
       '2020-06-12', '2020-06-13', '2020-06-14', '2020-06-15',
       '2020-06-16', '2020-06-17', '2020-06-18', '2020-06-19',
       '2020-06-20', '2020-06-21', '2020-06-22', '2020-06-23',
       '2020-06-24', '2020-06-25', '2020-06-26', '2020-06-27',
       '2020-06-28', '2020-06-29', '2020-06-30', '2020-07-01',
       '2020-07-02', '2020-07-03', '2020-07-04', '2020-07-05',
       '2020-07-06', '2020-07-07', '2020-07-08', '2020-07-09',
       '2020-07-10', '2020-07-11', '2020-07-12', '2020-07-13',
       '2020-07-14', '2020-07-15', '2020-07-16', '2020-07-17',
       '2020-07-18', '2020-07-19', '2020-07-20', '2020-07-21',
       '2020-07-22', '2020-07-23', '2020-07-24', '2020-07-25',
       '2020-07-26', '2020-07-27', '2020-07-28', '2020-07-29',
       '2020-07-30', '2020-07-31', '2020-08-01', '2020-

In [17]:
# Read shape file from shape_dir unsing geopandas
shape_dir = r'data/data/gadm36_CHE_1.shp'
shape_raw = gpd.read_file(shape_dir)

# New Section

In [18]:
shape_raw.head()

,GID_0,NAME_0,GID_1,NAME_1,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,geometry
0,CHE,Switzerland,CHE.1_1,Aargau,Argovia|Arg¢via|Argovie,None,Canton|Kanton|Chantun,Canton,None,CH.AG,"POLYGON ((7.94951 47.27751, 7.94897 47.27542, ..."
1,CHE,Switzerland,CHE.2_1,Appenzell Ausserrhoden,Appenzell Ausser-Rhoden|Appenzell Outer Rhodes...,None,Canton|Kanton|Chantun,Canton,None,CH.AR,"POLYGON ((9.20708 47.27694, 9.20828 47.27905, ..."
2,CHE,Switzerland,CHE.3_1,Appenzell Innerrhoden,Appenzell Inner-Rhoden|Appenzell Inner Rhodes|...,None,Canton|Kanton|Chantun,Canton,None,CH.AI,"MULTIPOLYGON (((9.51250 47.40100, 9.51430 47.4..."
3,CHE,Switzerland,CHE.4_1,Basel-Landschaft,Bâle-Campagne|Basel-Country|Baselland|Basel-La...,None,Canton|Kanton|Chantun,Canton,None,CH.BL,"MULTIPOLYGON (((7.36526 47.41939, 7.36377 47.4..."
4,CHE,Switzerland,CHE.5_1,Basel-Stadt,Bâle-Ville|Basel-City|Basel-Town|Basilea-Citad...,None,Canton|Kanton|Chantun,Canton,None,CH.BS,"POLYGON ((7.55254 47.54676, 7.55290 47.54829, ..."


In [19]:
# Extract canton name abbreviations from the attribute 'HASC_1', e.g. CH.AG --> AG, CH.ZH --> ZH
# And save into a new column named 'Canton' 
shape_raw['Canton'] = shape_raw['HASC_1'].str.replace('CH.', '')
canton_poly = shape_raw[['geometry','Canton']]


In [20]:
canton_poly.head()

,geometry,Canton
0,"POLYGON ((7.94951 47.27751, 7.94897 47.27542, ...",AG
1,"POLYGON ((9.20708 47.27694, 9.20828 47.27905, ...",AR
2,"MULTIPOLYGON (((9.51250 47.40100, 9.51430 47.4...",AI
3,"MULTIPOLYGON (((7.36526 47.41939, 7.36377 47.4...",BL
4,"POLYGON ((7.55254 47.54676, 7.55290 47.54829, ...",BS


In [21]:
demo_raw.shape

(27, 9)

In [22]:
merged_c_d = pd.merge(canton_poly, demo_raw, on='Canton')
merged_c_d.head()

,geometry,Canton,Population,SettlementAreaHa,SettlementAreaKm2,Density,O65,O65P,Beds,BedsPerCapita
0,"POLYGON ((7.94951 47.27751, 7.94897 47.27542, ...",AG,678207,23854.0,238.54,2843.158380,0.177,120043,1450,0.002138
1,"POLYGON ((9.20708 47.27694, 9.20828 47.27905, ...",AR,55234,2231.0,22.31,2475.750784,0.197,10881,208,0.003766
2,"MULTIPOLYGON (((9.51250 47.40100, 9.51430 47.4...",AI,16145,813.0,8.13,1985.854859,0.191,3084,18,0.001115
3,"MULTIPOLYGON (((7.36526 47.41939, 7.36377 47.4...",BL,288132,9025.0,90.25,3192.598338,0.219,63101,582,0.002020
4,"POLYGON ((7.55254 47.54676, 7.55290 47.54829, ...",BS,194766,2628.0,26.28,7411.187215,0.199,38758,1199,0.006156


In [23]:
merged_c_d.shape

(26, 10)

In [24]:
# merged_df = pd.merge(merged_c_d, canton_point, on='Canton')
merged_df = merged_c_d.merge(canton_point.rename(columns={'abbreviation_canton':'Canton'}))
merged_df

,geometry,Canton,Population,SettlementAreaHa,SettlementAreaKm2,Density,O65,O65P,Beds,BedsPerCapita,lat,long
0,"POLYGON ((7.94951 47.27751, 7.94897 47.27542, ...",AG,678207,23854.0,238.54,2843.158380,0.177,120043,1450,0.002138,47.449406,8.327495
1,"POLYGON ((9.20708 47.27694, 9.20828 47.27905, ...",AR,55234,2231.0,22.31,2475.750784,0.197,10881,208,0.003766,47.382710,9.271860
2,"MULTIPOLYGON (((9.51250 47.40100, 9.51430 47.4...",AI,16145,813.0,8.13,1985.854859,0.191,3084,18,0.001115,47.328414,9.409647
3,"MULTIPOLYGON (((7.36526 47.41939, 7.36377 47.4...",BL,288132,9025.0,90.25,3192.598338,0.219,63101,582,0.002020,47.482779,7.742975
4,"POLYGON ((7.55254 47.54676, 7.55290 47.54829, ...",BS,194766,2628.0,26.28,7411.187215,0.199,38758,1199,0.006156,47.558395,7.573271
5,"MULTIPOLYGON (((7.09284 46.89419, 7.09202 46.8...",BE,1034977,41197.0,411.97,2512.263029,0.208,215275,3053,0.002950,46.916667,7.466667
6,"MULTIPOLYGON (((6.78575 46.73603, 6.78559 46.7...",FR,318714,13998.0,139.98,2276.853836,0.157,50038,547,0.001716,46.718391,7.074008
7,"MULTIPOLYGON (((6.23440 46.33147, 6.23259 46.3...",GE,495249,9416.0,94.16,5259.653781,0.164,81221,1506,0.003041,46.195602,6.148113
8,"POLYGON ((8.90771 46.80860, 8.90603 46.81009, ...",GL,40403,1995.0,19.95,2025.213033,0.201,8121,86,0.002129,47.040570,9.068036
9,"MULTIPOLYGON (((10.22766 46.61207, 10.22734 46...",GR,198379,13863.0,138.63,1430.996177,0.213,42255,546,0.002752,46.796756,10.305946


In [25]:
daily_changes_df = pd.DataFrame() 
cases_per_canton_per_date = case_raw.iloc[: , 1:27].copy()

def calculate_daily_changes(data_toBe_treated):
  for row in range(data_toBe_treated.shape[0]):
    for col in range(data_toBe_treated.shape[1]):
      daily_changes_df.loc[row, col] = data_toBe_treated[row, col] - data_toBe_treated[row-1, col]


calculate_daily_changes(cases_per_canton_per_date.values)

daily_changes_df.columns = ['AG', 'AI', 'AR', 'BE', 'BL', 'BS', 'FR', 'GE', 'GL', 'GR', 
                            'JU', 'LU', 'NE', 'NW', 'OW', 'SG', 'SH', 'SO', 'SZ', 'TG', 'TI', 'UR', 'VD', 'VS', 'ZG', 'ZH']
daily_changes_df.index=dates

# daily_changes_df.fillna(0) # Fill null with zeros as well
daily_changes_df[0:1] = 0.0 # Fill null with zeros as well for first row only
daily_changes_df.tail()

,AG,AI,AR,BE,BL,BS,FR,GE,GL,GR,...,SH,SO,SZ,TG,TI,UR,VD,VS,ZG,ZH
2020-12-13,198.0,NaN,NaN,439.0,79.0,58.0,68.0,75.0,NaN,56.0,...,NaN,NaN,176.0,127.0,265.0,NaN,143.0,70.0,40.0,648.0
2020-12-14,305.0,NaN,NaN,312.0,100.0,59.0,92.0,171.0,NaN,217.0,...,NaN,NaN,0.0,197.0,174.0,NaN,291.0,116.0,37.0,503.0
2020-12-15,568.0,4.0,30.0,428.0,178.0,136.0,96.0,206.0,22.0,92.0,...,50.0,142.0,55.0,241.0,213.0,19.0,284.0,75.0,57.0,1003.0
2020-12-16,535.0,6.0,37.0,679.0,170.0,98.0,88.0,111.0,15.0,116.0,...,57.0,218.0,106.0,114.0,318.0,24.0,230.0,16.0,62.0,938.0
2020-12-17,NaN,8.0,51.0,613.0,150.0,119.0,85.0,NaN,56.0,123.0,...,40.0,178.0,108.0,0.0,258.0,12.0,NaN,98.0,63.0,1108.0


In [26]:
merged_df.shape, dates.shape, daily_changes_df.shape

((26, 12), (201,), (201, 26))

In [27]:
# For each date, extract a list of daily new cases per capita from all cantons(e.g. 'AG_diff_pc', 'AI_diff_pc', etc.), and add as a new column in merged
# For instance, in the column['2020-10-31'], there are: [0.0005411327220155498, nan, nan, 0.000496300306803826, ...]

for i,d in enumerate(dates):
  merged_df[d] = daily_changes_df.iloc[i].values

merged_df.tail()

,geometry,Canton,Population,SettlementAreaHa,SettlementAreaKm2,Density,O65,O65P,Beds,BedsPerCapita,...,2020-12-08,2020-12-09,2020-12-10,2020-12-11,2020-12-12,2020-12-13,2020-12-14,2020-12-15,2020-12-16,2020-12-17
21,"POLYGON ((8.48248 46.52972, 8.48077 46.53087, ...",UR,36433,2000.0,20.00,1821.650000,0.204,7432,63,0.001729,...,16.0,13.0,10.0,16.0,25.0,NaN,NaN,19.0,24.0,12.0
22,"POLYGON ((7.61459 45.97337, 7.61351 45.97326, ...",VS,343955,18463.0,184.63,1862.942100,0.196,67415,834,0.002425,...,288.0,325.0,280.0,246.0,196.0,143.0,291.0,284.0,230.0,NaN
23,"MULTIPOLYGON (((7.05791 46.86422, 7.05659 46.8...",VD,799145,29940.0,299.40,2669.154977,0.164,131060,2268,0.002838,...,78.0,142.0,116.0,122.0,72.0,70.0,116.0,75.0,16.0,98.0
24,"POLYGON ((8.66821 47.16368, 8.67379 47.16357, ...",ZG,126837,3306.0,33.06,3836.569873,0.170,21562,206,0.001624,...,41.0,52.0,52.0,47.0,63.0,40.0,37.0,57.0,62.0,63.0
25,"POLYGON ((8.87364 47.24979, 8.86491 47.24844, ...",ZH,1520968,37796.0,377.96,4024.150704,0.170,258565,4472,0.002940,...,1012.0,1056.0,938.0,1039.0,766.0,648.0,503.0,1003.0,938.0,1108.0


In [28]:
# Calculate circle sizes that are proportional to dnc per capita
# Set the latest dnc as default 
merged_df['size'] = merged_df.iloc[:,-1]*1e5/5+10
merged_df['dnc'] = merged_df.iloc[:,-1]

In [29]:
merged_df

,geometry,Canton,Population,SettlementAreaHa,SettlementAreaKm2,Density,O65,O65P,Beds,BedsPerCapita,...,2020-12-10,2020-12-11,2020-12-12,2020-12-13,2020-12-14,2020-12-15,2020-12-16,2020-12-17,size,dnc
0,"POLYGON ((7.94951 47.27751, 7.94897 47.27542, ...",AG,678207,23854.0,238.54,2843.158380,0.177,120043,1450,0.002138,...,463.0,415.0,379.0,198.0,305.0,568.0,535.0,NaN,NaN,NaN
1,"POLYGON ((9.20708 47.27694, 9.20828 47.27905, ...",AR,55234,2231.0,22.31,2475.750784,0.197,10881,208,0.003766,...,14.0,1.0,NaN,NaN,NaN,4.0,6.0,8.0,160010.0,160010.0
2,"MULTIPOLYGON (((9.51250 47.40100, 9.51430 47.4...",AI,16145,813.0,8.13,1985.854859,0.191,3084,18,0.001115,...,25.0,34.0,NaN,NaN,NaN,30.0,37.0,51.0,1020010.0,1020010.0
3,"MULTIPOLYGON (((7.36526 47.41939, 7.36377 47.4...",BL,288132,9025.0,90.25,3192.598338,0.219,63101,582,0.002020,...,619.0,556.0,605.0,439.0,312.0,428.0,679.0,613.0,12260010.0,12260010.0
4,"POLYGON ((7.55254 47.54676, 7.55290 47.54829, ...",BS,194766,2628.0,26.28,7411.187215,0.199,38758,1199,0.006156,...,192.0,189.0,170.0,79.0,100.0,178.0,170.0,150.0,3000010.0,3000010.0
5,"MULTIPOLYGON (((7.09284 46.89419, 7.09202 46.8...",BE,1034977,41197.0,411.97,2512.263029,0.208,215275,3053,0.002950,...,105.0,127.0,114.0,58.0,59.0,136.0,98.0,119.0,2380010.0,2380010.0
6,"MULTIPOLYGON (((6.78575 46.73603, 6.78559 46.7...",FR,318714,13998.0,139.98,2276.853836,0.157,50038,547,0.001716,...,103.0,112.0,98.0,68.0,92.0,96.0,88.0,85.0,1700010.0,1700010.0
7,"MULTIPOLYGON (((6.23440 46.33147, 6.23259 46.3...",GE,495249,9416.0,94.16,5259.653781,0.164,81221,1506,0.003041,...,128.0,146.0,99.0,75.0,171.0,206.0,111.0,NaN,NaN,NaN
8,"POLYGON ((8.90771 46.80860, 8.90603 46.81009, ...",GL,40403,1995.0,19.95,2025.213033,0.201,8121,86,0.002129,...,18.0,29.0,NaN,NaN,NaN,22.0,15.0,56.0,1120010.0,1120010.0
9,"MULTIPOLYGON (((10.22766 46.61207, 10.22734 46...",GR,198379,13863.0,138.63,1430.996177,0.213,42255,546,0.002752,...,139.0,86.0,78.0,56.0,217.0,92.0,116.0,123.0,2460010.0,2460010.0


In [30]:
merged_df.fillna(0)

,geometry,Canton,Population,SettlementAreaHa,SettlementAreaKm2,Density,O65,O65P,Beds,BedsPerCapita,...,2020-12-10,2020-12-11,2020-12-12,2020-12-13,2020-12-14,2020-12-15,2020-12-16,2020-12-17,size,dnc
0,"POLYGON ((7.94951 47.27751, 7.94897 47.27542, ...",AG,678207,23854.0,238.54,2843.158380,0.177,120043,1450,0.002138,...,463.0,415.0,379.0,198.0,305.0,568.0,535.0,0.0,0.0,0.0
1,"POLYGON ((9.20708 47.27694, 9.20828 47.27905, ...",AR,55234,2231.0,22.31,2475.750784,0.197,10881,208,0.003766,...,14.0,1.0,0.0,0.0,0.0,4.0,6.0,8.0,160010.0,160010.0
2,"MULTIPOLYGON (((9.51250 47.40100, 9.51430 47.4...",AI,16145,813.0,8.13,1985.854859,0.191,3084,18,0.001115,...,25.0,34.0,0.0,0.0,0.0,30.0,37.0,51.0,1020010.0,1020010.0
3,"MULTIPOLYGON (((7.36526 47.41939, 7.36377 47.4...",BL,288132,9025.0,90.25,3192.598338,0.219,63101,582,0.002020,...,619.0,556.0,605.0,439.0,312.0,428.0,679.0,613.0,12260010.0,12260010.0
4,"POLYGON ((7.55254 47.54676, 7.55290 47.54829, ...",BS,194766,2628.0,26.28,7411.187215,0.199,38758,1199,0.006156,...,192.0,189.0,170.0,79.0,100.0,178.0,170.0,150.0,3000010.0,3000010.0
5,"MULTIPOLYGON (((7.09284 46.89419, 7.09202 46.8...",BE,1034977,41197.0,411.97,2512.263029,0.208,215275,3053,0.002950,...,105.0,127.0,114.0,58.0,59.0,136.0,98.0,119.0,2380010.0,2380010.0
6,"MULTIPOLYGON (((6.78575 46.73603, 6.78559 46.7...",FR,318714,13998.0,139.98,2276.853836,0.157,50038,547,0.001716,...,103.0,112.0,98.0,68.0,92.0,96.0,88.0,85.0,1700010.0,1700010.0
7,"MULTIPOLYGON (((6.23440 46.33147, 6.23259 46.3...",GE,495249,9416.0,94.16,5259.653781,0.164,81221,1506,0.003041,...,128.0,146.0,99.0,75.0,171.0,206.0,111.0,0.0,0.0,0.0
8,"POLYGON ((8.90771 46.80860, 8.90603 46.81009, ...",GL,40403,1995.0,19.95,2025.213033,0.201,8121,86,0.002129,...,18.0,29.0,0.0,0.0,0.0,22.0,15.0,56.0,1120010.0,1120010.0
9,"MULTIPOLYGON (((10.22766 46.61207, 10.22734 46...",GR,198379,13863.0,138.63,1430.996177,0.213,42255,546,0.002752,...,139.0,86.0,78.0,56.0,217.0,92.0,116.0,123.0,2460010.0,2460010.0


In [31]:
import json
merged_df = merged_df.to_json()

In [32]:
# Build a GeoJSONDataSource from merged
geosource = GeoJSONDataSource(geojson = merged_df)
